# Setup

In [ ]:
!nvidia-smi

Tue May  2 22:41:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install torchsummaryX wandb --quiet


In [ ]:
import wandb

In [ ]:
# %cd TAPLoss-master/Demucs/denoiser

[Errno 2] No such file or directory: 'TAPLoss-master/Demucs/denoiser'
/content


In [ ]:
import os

os.environ["PREFIX"] = "wget -q https://cmu.box.com/shared/static"
os.environ["SUFFIX"] = "--content-disposition --show-progress"

!${PREFIX}/gwb1zmmrck3b5j5zbjpp303o6ij497at.zip ${SUFFIX}
!unzip -q 11785-spring23-projects.zip

```
11785-spring23-projects
├── eval
│   ├── radix
│   │   └── dns
│   │       └── source
│   │           ├── radix_dns_source_clean
│   │           └── radix_dns_source_noisy
│   └── relay
│       ├── slack
│       │   └── cloud
│       │       ├── src_slack_relay_auto
│       │       └── src_slack_relay_low
│       └── teams
│           ├── cloud
│           │   ├── src_teams_cloud_relay_auto
│           │   └── src_teams_cloud_relay_low
│           └── phone
│               ├── src_teams_phone_relay_auto
│               └── src_teams_phone_relay_low
└── train
    ├── radix
    │   └── dns
    │       └── source
    │           ├── radix_dns_source_clean
    │           └── radix_dns_source_noisy
    └── relay
        ├── slack
        │   └── cloud
        │       ├── relay_slack_auto
        │       └── relay_slack_low
        └── teams
            ├── cloud
            │   ├── relay_teams_cloud_high
            │   └── relay_teams_cloud_low
            └── phone
                ├── relay_teams_phone_high
                └── relay_teams_phone_low


```

In [ ]:
!wget https://github.com/YunyangZeng/TAPLoss/archive/refs/heads/master.zip
!unzip -q master.zip

!pip uninstall -y hydra_core hydra_colorlog pystoi git+https://github.com/ludlows/python-pesq#egg=pesq six sounddevice torchaudio torch 
!pip install -q julius hydra_core==0.11.3 hydra_colorlog==0.1.4 pystoi==0.3.3 git+https://github.com/ludlows/python-pesq#egg=pesq six==1.16.0 sounddevice torchaudio==0.13.1 torch==1.13.1

!wget -q https://cmu.box.com/shared/static/hv1xqf4m0iw86ffvtqrn9b4e2etg84nr --content-disposition --show-progress

!cp /content/TAPLoss-master/TAPLoss/* /content/TAPLoss-master/Demucs/denoiser

%cd /content/TAPLoss-master/Demucs/denoiser

Found existing installation: hydra-core 0.11.3
Uninstalling hydra-core-0.11.3:
  Successfully uninstalled hydra-core-0.11.3
Found existing installation: hydra-colorlog 0.1.4
Uninstalling hydra-colorlog-0.1.4:
  Successfully uninstalled hydra-colorlog-0.1.4
Found existing installation: pystoi 0.3.3
Uninstalling pystoi-0.3.3:
  Successfully uninstalled pystoi-0.3.3
Found existing installation: pesq 0.0.4
Uninstalling pesq-0.0.4:
  Successfully uninstalled pesq-0.0.4
Found existing installation: six 1.16.0
Uninstalling six-1.16.0:
  Successfully uninstalled six-1.16.0
Found existing installation: sounddevice 0.4.6
Uninstalling sounddevice-0.4.6:
  Successfully uninstalled sounddevice-0.4.6
Found existing installation: torchaudio 0.13.1
Uninstalling torchaudio-0.13.1:
  Successfully uninstalled torchaudio-0.13.1
Found existing installation: torch 1.13.1
Uninstalling torch-1.13.1:
  Successfully uninstalled torch-1.13.1
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolv

In [ ]:
%ls

conf/              launch_fine_tune.sh    setup.cfg
dataset/           make_debug.sh          setup.py
denoiser/          outputs/               TAP_estimator_model_v0.pt
egs/               paap_weights_v1.npy    TAP_estimator.py
img/               __pycache__/           TAPLoss.py
launch_debug.sh    README.md              train.py
launch_enhance.sh  requirements_cuda.txt  wandb/
launch_eval.sh     requirements.txt


# Initializing paths

In [ ]:
def remove_last_path(path_):
    return "/".join(path_.split("/")[:-1])

def make_training_config(
    training_voip_config_path ,
    train_noisy_path          ,
    test_noisy_path           ):

    train_noisy_path = remove_last_path(train_noisy_path)
    test_noisy_path = remove_last_path(test_noisy_path)
    training_voip_config_text = """
    #!/bin/bash

    dset:
      train: {}
      valid: {}
      test: {}
      noisy_json:
      noisy_dir:
      matching: dns
      eval_every: 1
    """.format(
        train_noisy_path,
        test_noisy_path,
        test_noisy_path
    )

    # print(training_voip_config_text)

    with open(training_voip_config_path, "w") as f:
        f.write(training_voip_config_text)

In [ ]:
def make_training_runner(
    training_voip_runner_path ,
    training_voip_config_path ,
    learning_rate             ,
    acoustic_loss_weight      ,
    stft_loss_weight          ,
    epochs                    ,
    num_workers               ,
    batch_size                ,
    continue_from             ,
    ddp                       ):
  
    descripton = training_voip_config_path.split("/")[-1].split(".")[-2]

    continue_pretrained = "dns64"
    dummy               = descripton
    dset                = descripton
    acoustic_loss       = "False" if acoustic_loss_weight == 0 else "True"
    acoustic_loss_only  = "False"
    phoneme_segmented   = "False"
    stft_loss           = "False" if stft_loss_weight == 0 else "True"
    ac_loss_weight      = acoustic_loss_weight
    lr = learning_rate
    acoustic_model_path = "/content/lld-estimation-model_12mse_14mae.pt"

    print(dset)
    training_voip_runner_text = """
    #!/bin/bash

    python train.py            \
        dummy={}               \
        continue_from={}       \
        continue_pretrained={} \
        dset={}                \
        acoustic_loss={}       \
        acoustic_loss_only={}  \
        stft_loss={}           \
        ac_loss_weight={}      \
        stft_loss_weight={}    \
        lr={}                  \
        epochs={}              \
        num_workers={}         \
        num_prints=1           \
        batch_size={}          \
        acoustic_model_path={} \
        ddp={} $@
    """.format(
        dummy               ,
        continue_from       ,
        continue_pretrained ,
        dset                ,
        acoustic_loss       ,
        acoustic_loss_only  ,
        stft_loss           ,
        ac_loss_weight      ,
        stft_loss_weight    ,
        learning_rate       ,
        epochs              ,
        num_workers         ,
        batch_size          ,
        acoustic_model_path ,
        ddp                 )

    with open(training_voip_runner_path, "w") as f:
        f.write(training_voip_runner_text)

    return training_voip_runner_text

In [ ]:
def make_clean_noisy_jsons(        
    train_noisy_path          ,
    test_noisy_path           ,
    train_clean_path          ,
    test_clean_path  ):

  os.system(f"python -m denoiser.audio {train_noisy_path} > {remove_last_path(train_noisy_path)}/noisy.json")
  os.system(f"python -m denoiser.audio {test_noisy_path} > {remove_last_path(test_noisy_path)}/noisy.json")
  os.system(f"python -m denoiser.audio {train_clean_path} > {remove_last_path(train_noisy_path)}/clean.json")
  os.system(f"python -m denoiser.audio {test_clean_path} > {remove_last_path(test_noisy_path)}/clean.json")

def setupTrainingRunner(
    training_voip_runner_path ,
    train_noisy_path          ,
    test_noisy_path           ,
    train_clean_path          ,
    test_clean_path           ,
    learning_rate             ,
    acoustic_loss_weight      ,
    stft_loss_weight          ,
    epochs                    ,
    num_workers               ,
    batch_size                ,
    continue_from             ,
    ddp                       ):

    training_voip_config_path = os.getcwd() + "/conf/dset/" + \
                                training_voip_runner_path.split("/")[-1]

    make_training_config(
        training_voip_config_path ,
        train_noisy_path          ,
        test_noisy_path           )
    
    make_clean_noisy_jsons(
        train_noisy_path          ,
        test_noisy_path           ,
        train_clean_path          ,
        test_clean_path  
    )

    training_voip_runner_text = make_training_runner(
        training_voip_runner_path ,
        training_voip_config_path ,
        learning_rate             ,
        acoustic_loss_weight      ,
        stft_loss_weight          ,
        epochs                    ,
        num_workers               ,
        batch_size                ,
        continue_from             ,
        ddp                       )

    return training_voip_runner_text

In [ ]:
def trainPartitions(
    training_runner_paths,
    train_noisy_paths    ,
    test_noisy_paths     ,
    train_clean_path     ,
    test_clean_path      ,
    learning_rate        ,
    acoustic_loss_weight ,
    stft_loss_weight     ,
    epochs               ,
    num_workers          ,
    batch_size           ,
    continue_from        ,
    ddp                  ):

    for (train_noisy_path, test_noisy_path, training_runner_path) in zip(train_noisy_paths, test_noisy_paths, training_runner_paths):

        training_voip_runner_text = setupTrainingRunner(
            training_runner_path ,
            base + train_noisy_path     ,
            base + test_noisy_path      ,
            base + train_clean_path     ,
            base + test_clean_path      ,
            learning_rate        ,
            acoustic_loss_weight ,
            stft_loss_weight     ,
            epochs               ,
            num_workers          ,
            batch_size           ,
            continue_from        ,
            ddp                  )
        os.environ["TRAINING_RUNNER_PATH"] = training_runner_path
        !sh {training_runner_path}
        break


In [ ]:
base = "/content/11785-spring23-projects/"

train_clean_path =\
    "train/radix/dns/source/radix_dns_source_clean"

test_clean_path =\
    "eval/radix/dns/source/radix_dns_source_clean"

train_noisy_paths = [
    "train/relay/slack/cloud/relay_slack_auto"  ,
    "train/relay/slack/cloud/relay_slack_low"   ]

test_noisy_paths = [
    "eval/relay/slack/cloud/src_slack_relay_auto"  ,
    "eval/relay/slack/cloud/src_slack_relay_low"   ]

training_runner_paths = [base + name + ".yaml" for name in train_noisy_paths]

# Wandb Settings

In [ ]:
wandb.login(key="b5dcd1a0d394d9fdfb6c790ec846e0613a539137", relogin=True) #API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# Moving the wandb section to solver.py inside Demucs/Denoiser where training loop takes place

# config = {
#     'learning_rate'        : 3e-4,
# 'acoustic_loss_weight' : 0.0,
# 'stft_loss_weight'     : 1.0,
# 'epochs'               : 10,
# 'num_workers'          : 2,
# 'batch_size'           : 12,
# 'ddp'                  : 1,
# 'continue_from'        : "",
# 'optimizer': "adam"
#     # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
# }

# Training Ablations

In [ ]:

learning_rate        = 8e-4
acoustic_loss_weight = 0.0
stft_loss_weight     = 1.0
epochs               = 5
num_workers          = 2
batch_size           = 10
ddp                  = 1
continue_from        = ""

trainPartitions(
    training_runner_paths,
    train_noisy_paths    ,
    test_noisy_paths     ,
    train_clean_path     ,
    test_clean_path      ,
    learning_rate        ,
    acoustic_loss_weight ,
    stft_loss_weight     ,
    epochs               ,
    num_workers          ,
    batch_size           ,
    continue_from        ,
    ddp                  )

relay_slack_auto
args {'sample_rate': 16000, 'segment': 10, 'stride': 2, 'pad': True, 'remix': False, 'bandmask': 0.0, 'shift': 16000, 'shift_same': True, 'revecho': 0, 'num_prints': 1, 'device': 'cuda', 'num_workers': 2, 'verbose': 0, 'show': 0, 'checkpoint': True, 'continue_from': None, 'continue_best': False, 'continue_pretrained': 'dns64', 'restart': False, 'save_checkpoints': True, 'checkpoint_file': 'checkpoint.th', 'best_file': 'best.th', 'history_file': 'history.json', 'samples_dir': 'samples', 'save_again': False, 'seed': 2022, 'dummy': 'relay_slack_auto', 'pesq': True, 'eval_every': 1, 'dry': 0.0, 'streaming': False, 'optim': 'adam', 'lr': 0.0008, 'gradient_clip': True, 'beta2': 0.999, 'loss': 'l1', 'grad_max_norm': 0.01, 'stft_loss': True, 'acoustic_model_path': '/content/lld-estimation-model_12mse_14mae.pt', 'acoustic_loss': False, 'acoustic_loss_only': False, 'ac_loss_weight': 0.0, 'ac_loss_type': 'frame_energy_weighted_l1', 'stft_sc_factor': 0.5, 'stft_mag_factor': 0.5, '

In [ ]:

learning_rate        = 3e-4
acoustic_loss_weight = 0.3
stft_loss_weight     = 0.7
epochs               = 20
num_workers          = 2
batch_size           = 12
ddp                  = 1
continue_from        = ""

trainPartitions(
    training_runner_paths,
    train_noisy_paths    ,
    test_noisy_paths     ,
    train_clean_path     ,
    test_clean_path      ,
    learning_rate        ,
    acoustic_loss_weight ,
    stft_loss_weight     ,
    epochs               ,
    num_workers          ,
    batch_size           ,
    continue_from        ,
    ddp                  )

relay_slack_auto
args {'sample_rate': 16000, 'segment': 10, 'stride': 2, 'pad': True, 'remix': False, 'bandmask': 0.0, 'shift': 16000, 'shift_same': True, 'revecho': 0, 'num_prints': 1, 'device': 'cuda', 'num_workers': 2, 'verbose': 0, 'show': 0, 'checkpoint': True, 'continue_from': None, 'continue_best': False, 'continue_pretrained': 'dns64', 'restart': False, 'save_checkpoints': True, 'checkpoint_file': 'checkpoint.th', 'best_file': 'best.th', 'history_file': 'history.json', 'samples_dir': 'samples', 'save_again': False, 'seed': 2022, 'dummy': 'relay_slack_auto', 'pesq': True, 'eval_every': 1, 'dry': 0.0, 'streaming': False, 'optim': 'adam', 'lr': 0.0003, 'gradient_clip': True, 'beta2': 0.999, 'loss': 'l1', 'grad_max_norm': 0.01, 'stft_loss': True, 'acoustic_model_path': '/content/lld-estimation-model_12mse_14mae.pt', 'acoustic_loss': True, 'acoustic_loss_only': False, 'ac_loss_weight': 0.3, 'ac_loss_type': 'frame_energy_weighted_l1', 'stft_sc_factor': 0.5, 'stft_mag_factor': 0.5, 's

# Generating enhanced files

In [ ]:
%mkdir ../../../11785-spring23-projects/eval/relay/slack/cloud/enhanced

In [ ]:
# Generate enhanced files from trainined model and store them into ../enhan
!python -m denoiser.enhance --model_path=/content/TAPLoss-master/Demucs/denoiser/outputs/best.th --num_workers 4 --noisy_json ../../../11785-spring23-projects/eval/relay/slack/cloud/noisy.json --out_dir ../../../11785-spring23-projects/eval/relay/slack/cloud/enhanced

INFO:denoiser.pretrained:Loading model from /content/TAPLoss-master/Demucs/denoiser/outputs/exp_ac_loss_weight=0.5,acoustic_loss=True,acoustic_loss_only=False,acoustic_model_path=/content/lld-estimation-model_12mse_14mae.pt,batch_size=12,continue_pretrained=dns64,dset=relay_slack_auto,dummy=relay_slack_auto,epochs=20,lr=0.0003,stft_loss=True,stft_loss_weight=0.5/best.th
../../../11785-spring23-projects/eval/relay/slack/cloud/enhanced
INFO:__main__:Generate enhanced files | 30/150 | 227.5 it/sec
INFO:__main__:Generate enhanced files | 60/150 | 273.6 it/sec
INFO:__main__:Generate enhanced files | 90/150 | 272.4 it/sec
INFO:__main__:Generate enhanced files | 120/150 | 295.0 it/sec
INFO:__main__:Generate enhanced files | 150/150 | 283.4 it/sec
Waiting for pending jobs...
INFO:__main__:Generate enhanced files | 30/150 | 1.3 it/sec
INFO:__main__:Generate enhanced files | 60/150 | 1.2 it/sec
INFO:__main__:Generate enhanced files | 90/150 | 1.3 it/sec
INFO:__main__:Generate enhanced files | 12

In [ ]:
# Remove noisy files from enhnaced
!ls
# %cd ../..
# %cd /content/11785-spring23-projects/eval/relay/slack/cloud/enhanced
! rm -r *_noisy.wav

clnsp102_traffic_248091_3_snr0_tl-21_fileid_268_enhanced.wav
clnsp102_traffic_248091_3_snr0_tl-21_fileid_268_noisy.wav
clnsp104_G3hs6PFniM0_snr4_tl-27_fileid_133_enhanced.wav
clnsp104_G3hs6PFniM0_snr4_tl-27_fileid_133_noisy.wav
clnsp113_car_19980_0_snr18_tl-30_fileid_178_enhanced.wav
clnsp113_car_19980_0_snr18_tl-30_fileid_178_noisy.wav
clnsp126_3Wjw0nadnM4_snr15_tl-22_fileid_0_enhanced.wav
clnsp126_3Wjw0nadnM4_snr15_tl-22_fileid_0_noisy.wav
clnsp129_lJoaywrZPsU_snr0_tl-27_fileid_252_enhanced.wav
clnsp129_lJoaywrZPsU_snr0_tl-27_fileid_252_noisy.wav
clnsp146_baby_416657_0_snr9_tl-25_fileid_72_enhanced.wav
clnsp146_baby_416657_0_snr9_tl-25_fileid_72_noisy.wav
clnsp148_4AGxmFdExYc_snr0_tl-28_fileid_238_enhanced.wav
clnsp148_4AGxmFdExYc_snr0_tl-28_fileid_238_noisy.wav
clnsp149_SjEWn2DhLDs_snr0_tl-23_fileid_58_enhanced.wav
clnsp149_SjEWn2DhLDs_snr0_tl-23_fileid_58_noisy.wav
clnsp153_bus_109303_3_snr13_tl-34_fileid_197_enhanced.wav
clnsp153_bus_109303_3_snr13_tl-34_fileid_197_noisy.wav
clnsp